In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import re

In [8]:
def get_content(url):
    headers = {
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding":"gzip, deflate, sdch",
        "Accept-Language":"ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4",
        "Cache-Control":"max-age=0",
        "Connection":"keep-alive",
        "Cookie":"npic=eFBEBBU/yzEme0B5l5OcagDCMtaa5/G90XoG9Rkq9EP/wNGQGt+jPKJaBOveBEaFCA==; NNB=NLTF6ONPDQTFO; nx_ssl=2; ncvid=#vid#_14.39.150.134yORl; nci4=1e2acde1f7bcd6663bd59285887a057cdd867892c69d6fa17cb6ad5dd448b513813ffc27a629b6bcec4005d665c64d9647474a6bb086e7e8fc09e14d839409f9b7afa285a495d9a9a88faa9bd3a0b093b789c4b5bc9bbe8f978f82a286b7f6f2f3f2efededed9a95b097a9e4e08ce29a9bf28697f88994e7; nid_iplevel=1; page_uid=xERzTlpyLPZss5rLUkNssssssSh-406270; ncvc2=b4d42c085c6a6bdced107b5d7f9de2853a6a933c5f16c954b5704d9e2200e62df804c21bd9e570582abfe699; JSESSIONID=DD056961ED5BA6436D706429CBF42060; BMR=s=1464245784249&r=http%3A%2F%2Fm.cafe.naver.com%2FArticleRead.nhn%3Fclubid%3D10050146%26menuid%3D334%26articleid%3D319943171%26query%3D%25EB%25A7%25A5%25EB%25B6%2581&r2=http%3A%2F%2Fm.cafe.naver.com%2FArticleSearchList.nhn%3Fsearch.query%3D%25EB%25A7%25A5%25EB%25B6%2581%26search.menuid%3D%26search.searchBy%3D0%26search.sortBy%3Dsim%26search.clubid%3D10050146",
        "Host":"m.cafe.naver.com",
        "Referer":"http://m.cafe.naver.com/ArticleSearchList.nhn?search.query=%EB%A7%A5%EB%B6%81&search.menuid=&search.searchBy=0&search.sortBy=sim&search.clubid=10050146",
        "Upgrade-Insecure-Requests":"1",
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36",
    }
    response = requests.get(url, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    return dom.select_one("#postContent").text

In [3]:
def crawl_content(url):
    try:
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        return dom.select_one("#postContent").text
    except:
        return ""

In [4]:
def find_phonenumber(content):
    pattern = re.compile("[0-9O공영일이둘삼셋사넷오육칠팔구]{3}[.-]?[0-9O공영일이둘삼셋사넷오육칠팔구]{3,4}[.-]?[0-9O공영일이둘삼셋사넷오육칠팔구]{4}")
    phonenumbers = pattern.findall(content)
    if phonenumbers:
        return phonenumbers[0]
    return ""
def preprocess_phonenumber(phonenumber):
    return phonenumber.replace("삼","3").replace("O","0").replace("칠","7").replace("구","9")

In [5]:
url = "http://m.cafe.naver.com/ArticleSearchList.nhn?search.query=맥북&search.menuid=&search.searchBy=0&search.sortBy=sim&search.clubid=10050146"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) " + \
            "Chrome/37.0.2062.94 Safari/537.36"
headers = {"User-Agent": user_agent}

response = requests.get(url, headers=headers)
dom = BeautifulSoup(response.content, "html.parser")
rows = dom.select("#articleList li")

df = pd.DataFrame(columns=["title","writer","no","date","link","content", "phonenumber"])

for row in rows:
    link = "http://m.cafe.naver.com" + row.select_one("a").get("href")
    title = row.select_one("h3").text.strip()
    writer = row.select_one(".name").text
    date = row.select_one(".time").text
    no = row.select_one(".no em").text
    content = get_content(link)
    phonenumber = preprocess_phonenumber(find_phonenumber(content))
    
    data_dict = {
        "title":title, 
        "writer":writer, 
        "no":no, 
        "date":date, 
        "link":link, 
        "content":content, 
        "phonenumber":phonenumber
    }

    df.loc[len(df)] = data_dict    
    
df.head()

http://m.cafe.naver.com/ArticleRead.nhn?clubid=10050146&menuid=334&articleid=327427658&query=맥북 

      중고나라 공식 앱 다운받기      편한 택배 / 퀵 신청하기 판 매 양 식아이디이메일싸이,블로그,타카페,타사이트 링크시 삭제 및 강퇴거주지역도,시,동까지 정확히 기재판매 제품명구입시기년,월 기재희망가격정확히 기재: (3만~4만등의 경매 유도글 삭제)거래방법방문택배 신청하기, 직거래, 택배,  안전거래상세설명최초 구매해 지금껏 사용해 온 맥북입니다.  구입 초기 이후 사용빈도가 많지는 않습니다.  사진에 보이듯이... 큰 상처나 크랙등은 없고 세월 정도의 사용감 있습니다.   램 및 OS 스노우 레오파드 구입 후 업그레이드 하였으며 판매를 위해 초기화했습니다. 테스트 결과 블루투스, USB 등 전 기능 이상없이 잘 작동되는 것 확인하였습니다.  다만 1. 배터리 방전 상태로 전원 꼽아 사용하셔야 합니다. (아님 이베이에서 호환 배터리 구입 권유드립니다) 2. CD, DVD롬의 경우 설치 디스크, DVD등 경우 예외없이 잘 작동되나 음악 CD의 경우 반 정도를 뱉어냅니다. 클리닝 CD 한번이면 깨끗해질 것도 같은데... 아무튼 감안해서 매입 부탁드립니다. 3. 전원 연결 코드 및 어댑터가 좀 지저분합니다. 이외에는 모두 정상인 듯 합니다. 테스트를 위해 개봉해 본 리모컨, OS, 최초 램 등 포함하여 사진에 보이는 물품 전부 포함이며11만원에 네고없이 노원구 부근에서 직거래만 하겠습니다.맥북을 사용해보신 분께서 구입하셨음 좋겠습니다. 위 항목을 (거래금지 항목 포함) 허위작성하거나 임의 삭제 또는 채우지 않을 경우 무통보 삭제될수 있습니다.          중고나라 공식 앱 다운받기      편한 택배 / 퀵 신청하기 판 매 양 식아이디이메일싸이,블로그,타카페,타사이트 링크시 삭제 및 강퇴거주지역도,시,동까지 정확히 기재판매 제품명구입시기년,월 기재희망가격정확히 기재: (3만~4

,title,writer,no,date,link,content,phonenumber
0,구형 맥북 화이트 A1181 판매합니다,채린아빠,42,12:50,http://m.cafe.naver.com/ArticleRead.nhn?clubid...,\n\n 중고나라 공식 앱 다운받기 편한 택배 / 퀵 신청하기 판...,
1,음악작업용 고사양 맥북프로 2.8 i7/10GB/Fusion750GB (Komple...,ssimba1204,210,16.07.12.,http://m.cafe.naver.com/ArticleRead.nhn?clubid...,"\n\n ·2011 Late Macbook Pro 13"" 고급형CPU:2.8Ghz ...",0104040449육
2,인케이스 ICON Sleeve with TENSAERLITE 파우치 맥북 프로 레티...,DODOBOO,112,16.07.05.,http://m.cafe.naver.com/ArticleRead.nhn?clubid...,\n\n 중고나라 공식 앱 다운받기 편한 택배 / 퀵 신청하기 판...,010-팔팔18-71육2
3,2016 early 뉴맥북 (MacBook) 기본형 판매합니다 (스페이스 그레이 색상),lanchert,257,16.07.04.,http://m.cafe.naver.com/ArticleRead.nhn?clubid...,\n\n 중고나라 공식 앱 다운받기 편한 택배 / 퀵 신청하기 판...,
4,맥북 알루미늄 유니바디13인치 (2009) 팝니다.,디언,180,16.07.03.,http://m.cafe.naver.com/ArticleRead.nhn?clubid...,\n\n 중고나라 공식 앱 다운받기 편한 택배 / 퀵 신청하기 판...,
